# RNA-Seq Analysis using Snakemake and AWS Batch

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called [Nextflow](https://www.nextflow.io) run via [AWS Batch](https://aws.amazon.com/batch/). If you completed the other tutorials in this repo, you will see that it is similar to Tutorial 2, but instead of running Snakemake locally, we switch to Nextflow and run it using Batch in a serverless manner. 

Before begining this tutorial, if you do not have required roles, policies, permissions or compute environment please click [here](https://github.com/STRIDES/NIHCloudLabAWS/blob/zbyosufzai-awsbatch-1/notebooks/AWSBatch/Intro_AWS_Batch.ipynb#install_nextflow) to set that up.

## Prerequisites
#### Python requirements
+ Python >= 3.8

#### AWS requirements
+ Please ensure you have a VPC, subnets, and security group set up before running this tutorial.
+ Role with AdministratorAccess, AmazonSageMakerFullAccess, S3 access and AWSBatchServiceRole.
+ Instance Role with AmazonECS_FullAccess, AmazonEC2ContainerRegistryFullAccess, and S3 access.
+ If you do not have the required set-up for AWS Batch please follow this tutorial [here](https://github.com/STRIDES/NIHCloudLabAWS/blob/zbyosufzai-awsbatch-1/notebooks/AWSBatch/Intro_AWS_Batch.ipynb#install_nextflow). ***When making the instance role, make another for SageMaker notebooks with the following permissions: AdminstratorAccess, AmazonEC2ContainerRegistryFullAccess, AmazonECS_FullAccess, AmazonS3FullAccess, AmazonSageMakerFullAccess, and AWSBatchServiceRole.***
It is recommended that specific permission to folders are added through inline policy. An example of the JSON is below:

<pre>
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowSageMakerS3Access",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:CreateBucket"
            ],
            "Resource": [
                "arn:aws:s3:::batch-bucket",
                "arn:aws:s3:::batch-bucket/*",
                "arn:aws:s3:::nigms-sandbox-healthomics",
                "arn:aws:s3:::nigms-sandbox-healthomics/*",
                "arn:aws:s3:::ngi-igenomes",
                "arn:aws:s3:::ngi-igenomes/*"
            ]
        }
    ]
}
</pre>
For AWS bucket naming conventions, please click [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/bucketnamingrules.html).

## Getting Started
If you would to skip building in the cloud and just launch the stack please click the button below and follow the instructions to complete stack deployment. The deployment should take ~5 min and then the resources will be ready for use.


[![Launch Stack](images/LaunchStack.jpg)](https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/new?stackName=aws-batch-nigms&templateURL=https://cf-templates-w389nf9f2fxs-us-east-1.s3.us-east-1.amazonaws.com/AWSBatch_template.yaml)

Select Next at the bottom right. ![Launch Stack Select](images/LaunchStack_select.png)

Select Next again at the bottom right of the screen. ![Launch Stack Specify](images/LaunchStack_specify.png)

Select I acknowledge that AWS CloudFormation might create IAM resources and Next. ![Launch Stack Configure](images/LaunchStack_configure.png)

Review and select Submit. ![Launch Stack Review](images/LaunchStack_review.png)

When creating the SageMaker notebook please select the corresponding IAM role. The role name will start with aws-batch-nigms-SageMakerExecutionrole-*unique_identifier*. ![SageMaker IAM Role](images/SageMaker_IAM_Role.png)

### Step 1. Install required dependencies, update paths and create a new S3 Bucket to store input and output files (if needed)


In [ ]:
import boto3
# Get account ID and region 
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

In [ ]:
# Set variable names 
# These variables should come from the Intro AWS Batch tutorial (or leave as-is if using the launch stack button)
BUCKET_NAME = "aws-batch-nigms-batch-bucket-" + account_id
INPUT_FOLDER = 'nigms-sandbox/me-inbre-rnaseq-pipelinev2'
AWS_QUEUE = 'aws-batch-nigms-JobQueue'
AWS_REGION = region

In [ ]:
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge

In [ ]:
# Add to your path, do this every time you restart your kernel
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

In [ ]:
# Install Nextflow
! mamba install -y -c conda-forge -c bioconda nextflow

In [ ]:
##### Import relevant libraries
# Created using this https://github.com/STRIDES/NIHCloudLabAWS/blob/zbyosufzai-awsbatch-1/notebooks/AWSBatch/Intro_AWS_Batch.ipynb#install_nextflow
#Run if you don't have Java installed
! sudo apt update
! sudo apt-get install default-jdk -y
! java -version

In [ ]:
#Install nexflow, make it exceutable, and update it
! curl https://get.nextflow.io | bash
! chmod +x nextflow
! ./nextflow self-update

In [ ]:
# replace batch bucket name in nextflow configuration file
! sed -i "s/aws-batch-nigms-batch-bucket-/$BUCKET_NAME/g" nextflow.config

In [ ]:
# replace the region placeholder with the region you are in 
! sed -i "s/aws-region/$AWS_REGION/g" nextflow.config

### Step 2. Enable AWS Batch for the nextflow script 

In [ ]:
# Run nextflow script with parameters 
! ./nextflow run main.nf --input s3://$INPUT_FOLDER/data/raw_fastqSub/samplesheet.csv -profile docker,awsbatch -c nextflow.config --awsqueue  $AWS_QUEUE --awsregion $AWS_REGION

### STEP 3: Report the top 10 most highly expressed genes in the samples.

In [ ]:
# View output files in folder
! aws s3 ls s3://$BUCKET_NAME/nextflow_output/data/quant --recursive | cut -c32-

In [ ]:
# Copy output to here 
! aws s3 sync s3://$BUCKET_NAME/nextflow_output/data/quant quant

View the top 10 most highly expressed genes in the double lysogen sample.

In [ ]:
%%bash
for samp in quant/*/quant.sf; 
    do echo $samp; 
    sort -nrk 5,5 quant/*/quant.sf | head -10; 
    done

### STEP 4: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
%%bash
for samp in quant/*/quant.sf; do echo $samp; 
    echo Name    Length  EffectiveLength TPM     NumReads;
    grep 'BB28_RS16545' quant/*/quant.sf; 
    done